# Using labels for orchestration and automation

1. Label nodes
1. Deploy pods only to labeled nodes
1. Query pods by labels 

In [2]:
!k3d create --publish 80:80 --workers 3

2019/06/23 18:28:55 Created cluster network with ID cbfc510280bd837824395acd953f02d0c298e60b2f0c0defe37e09932cafda39
2019/06/23 18:28:55 Creating cluster [k3s-default]
2019/06/23 18:28:55 Creating server using docker.io/rancher/k3s:v0.5.0...
2019/06/23 18:28:56 Booting 3 workers for cluster k3s-default
2019/06/23 18:28:56 Created worker with ID c681c5bba823cb616e96e85b9c1ae652248ffb7ef583da8b538b582f8260363c
2019/06/23 18:28:57 Created worker with ID 41a32efb9b1eac6e2e09e174bed4617e5ee01eaffeb51d72d32284a79c366132
2019/06/23 18:28:58 Created worker with ID 90577eff4ea5a003caf8c7d2f490465fdc5fbb472ba99f81a77b25dc7f62c772
2019/06/23 18:28:58 SUCCESS: created cluster [k3s-default]
2019/06/23 18:28:58 You can now use the cluster with:

export KUBECONFIG="$(k3d get-kubeconfig --name='k3s-default')"
kubectl cluster-info


In [4]:
!cp "$(k3d get-kubeconfig --name='k3s-default')" ~/.kube/config

Test ingress like this example
 https://github.com/rancher/k3d/issues/11#issuecomment-492822086

In [5]:
%%bash
cat <<EOF | kubectl create -f  -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: whoami
  labels:
    app: whoami
spec:
  replicas: 4
  selector:
    matchLabels:
      app: whoami
  template:
    metadata:
      labels:
        app: whoami
    spec:
      containers:
      - name: whoami
        image: containous/whoami:latest
        ports:
        - containerPort: 80
EOF

deployment.apps/whoami created


query deployments unttill available becomes "2"

In [7]:
!kubectl get deployments

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
whoami   4/4     4            4           5s


In [12]:
!kubectl get pods -o wide

NAME                      READY   STATUS    RESTARTS   AGE    IP          NODE                       NOMINATED NODE   READINESS GATES
whoami-6d996cf96f-44cxv   1/1     Running   0          107s   10.42.0.4   k3d-k3s-default-server     <none>           <none>
whoami-6d996cf96f-4njp2   1/1     Running   0          107s   10.42.3.3   k3d-k3s-default-worker-2   <none>           <none>
whoami-6d996cf96f-hq6qx   1/1     Running   0          107s   10.42.1.4   k3d-k3s-default-worker-0   <none>           <none>
whoami-6d996cf96f-xrdbj   1/1     Running   0          107s   10.42.2.4   k3d-k3s-default-worker-1   <none>           <none>


In [19]:
%%bash
cat <<EOF | kubectl apply -f  -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: whoami-restricted
  labels:
    app: whoami-restricted
spec:
  replicas: 2
  selector:
    matchLabels:
      app: whoami-restricted
  template:
    metadata:
      labels:
        app: whoami-restricted

    spec:
      containers:
      - name: whoami
        image: containous/whoami:latest
        ports:
        - containerPort: 80
      nodeSelector:
          placement: other
EOF

deployment.apps/whoami-restricted created


In [25]:
!kubectl get pods -o wide

NAME                                 READY   STATUS    RESTARTS   AGE   IP          NODE                       NOMINATED NODE   READINESS GATES
whoami-6d996cf96f-44cxv              1/1     Running   0          10m   10.42.0.4   k3d-k3s-default-server     <none>           <none>
whoami-6d996cf96f-4njp2              1/1     Running   0          10m   10.42.3.3   k3d-k3s-default-worker-2   <none>           <none>
whoami-6d996cf96f-hq6qx              1/1     Running   0          10m   10.42.1.4   k3d-k3s-default-worker-0   <none>           <none>
whoami-6d996cf96f-xrdbj              1/1     Running   0          10m   10.42.2.4   k3d-k3s-default-worker-1   <none>           <none>
whoami-restricted-84b895f667-tgnhl   0/1     Pending   0          2m    <none>      <none>                     <none>           <none>
whoami-restricted-84b895f667-xn2m8   0/1     Pending   0          2m    <none>      <none>                     <none>           <none>


In [24]:
!kubectl describe deployments whoami-restricted

Name:                   whoami-restricted
Namespace:              default
CreationTimestamp:      Sun, 23 Jun 2019 18:40:37 +0200
Labels:                 app=whoami-restricted
Annotations:            deployment.kubernetes.io/revision: 1
                        kubectl.kubernetes.io/last-applied-configuration:
                          {"apiVersion":"apps/v1","kind":"Deployment","metadata":{"annotations":{},"labels":{"app":"whoami-restricted"},"name":"whoami-restricted","...
Selector:               app=whoami-restricted
Replicas:               2 desired | 2 updated | 2 total | 0 available | 2 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=whoami-restricted
  Containers:
   whoami:
    Image:        containous/whoami:latest
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type         

We are stuck with deployment in a "ScalingReplicaSet". This is because there is no node marked as "placement: other":

In [27]:
!kubectl get nodes -l placement=other

No resources found.


In [28]:
!kubectl label nodes k3d-k3s-default-worker-2 placement=other

node/k3d-k3s-default-worker-2 labeled


In [29]:
!kubectl get pods -o wide

NAME                                 READY   STATUS    RESTARTS   AGE     IP          NODE                       NOMINATED NODE   READINESS GATES
whoami-6d996cf96f-44cxv              1/1     Running   0          12m     10.42.0.4   k3d-k3s-default-server     <none>           <none>
whoami-6d996cf96f-4njp2              1/1     Running   0          12m     10.42.3.3   k3d-k3s-default-worker-2   <none>           <none>
whoami-6d996cf96f-hq6qx              1/1     Running   0          12m     10.42.1.4   k3d-k3s-default-worker-0   <none>           <none>
whoami-6d996cf96f-xrdbj              1/1     Running   0          12m     10.42.2.4   k3d-k3s-default-worker-1   <none>           <none>
whoami-restricted-84b895f667-tgnhl   1/1     Running   0          4m36s   10.42.3.5   k3d-k3s-default-worker-2   <none>           <none>
whoami-restricted-84b895f667-xn2m8   1/1     Running   0          4m36s   10.42.3.4   k3d-k3s-default-worker-2   <none>           <none>


Once node was labeled, K8s scheduler assinged both pods to this node

In [30]:
!kubectl get nodes -l placement=other

NAME                       STATUS   ROLES    AGE   VERSION
k3d-k3s-default-worker-2   Ready    <none>   16m   v1.14.1-k3s.4


In [32]:
# node label does not propagate to pod
!kubectl get pods -l placement=other

No resources found.


In [37]:
!kubectl get pods --show-labels=true

NAME                                 READY   STATUS    RESTARTS   AGE   LABELS
whoami-6d996cf96f-44cxv              1/1     Running   0          28m   app=whoami,pod-template-hash=6d996cf96f
whoami-6d996cf96f-4njp2              1/1     Running   0          28m   app=whoami,pod-template-hash=6d996cf96f
whoami-6d996cf96f-hq6qx              1/1     Running   0          28m   app=whoami,pod-template-hash=6d996cf96f
whoami-6d996cf96f-xrdbj              1/1     Running   0          28m   app=whoami,pod-template-hash=6d996cf96f
whoami-restricted-84b895f667-tgnhl   1/1     Running   0          20m   app=whoami-restricted,pod-template-hash=84b895f667
whoami-restricted-84b895f667-xn2m8   1/1     Running   0          20m   app=whoami-restricted,pod-template-hash=84b895f667


In [99]:
# delete cluster
!k3d d 

2019/06/22 18:17:48 Removing cluster [k3s-default]
2019/06/22 18:17:48 ...Removing 2 workers
2019/06/22 18:17:50 ...Removing server
2019/06/22 18:17:52 SUCCESS: removed cluster [k3s-default]
